# Packages

In [1]:
import ee
import geemap

# ESA WorldCover

The ESA WorldCover 10m 2020 product provides a global land cover map for 2020 at 10m resolution based on Sentinel-1 and Sentinel-2 data. The product comes with 11 land cover classes and has been generated in the framework of the ESA WorldCover project, part of the 5th Earth Observation Envelop Programme (EOEP-5) of the SA.

## Using Web Map Services

The product can also be used within other websites or GIS clients by Web Map Services. These provide a direct link to cached images and are the best option if you want to map data and produce cartographic products. They are not suitable for analysis, as the data are represented only as RGB images.
- WMTS: https://services.terrascope.be/wmts/v2
- WMS: https://services.terrascope.be/wms/v2
- Layers: WORLDCOVER_2020_MAP, WORLDCOVER_2020_S2_FCC, WORLDCOVER_2020_S2_TCC

In [2]:
Map = geemap.Map()

esa_wms = 'https://services.terrascope.be/wms/v2' # WMS url
tcc_layer = 'WORLDCOVER_2020_S2_TCC' # True color composite
fcc_layer = 'WORLDCOVER_2020_S2_FCC' # False color composite
map_layer = 'WORLDCOVER_2020_MAP' # Land cover classification map

Map.add_wms_layer(esa_wms, layers = tcc_layer, name = 'True Color', attribution = 'ESA')
Map.add_wms_layer(esa_wms, layers = fcc_layer, name = 'False Color', attribution = 'ESA')
Map.add_wms_layer(esa_wms, layers = map_layer, name = 'Classificaiton', attribution = 'ESA')

Map.add_legend(title = 'ESA Land Cover', builtin_legend = 'ESA_WorldCover')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Using Earth Engine

In [3]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

esa = ee.ImageCollection('ESA/WorldCover/v100').first()
esa_vis = {'bands': ['Map']}

Map.addLayer(esa, esa_vis, 'ESA Land Cover')
Map.add_legend(title = 'ESA Land Cover', builtin_legend = 'ESA_WorldCover')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Creating charts

In [4]:
histogram = geemap.image_histogram(
    esa, scale = 1000, x_label = 'Land Cover Type', y_label = 'Area (km2)'
)
histogram

In [5]:
df = geemap.image_histogram(esa, scale = 1000, return_df = True)
df

,key,value
0,10,6.139810e+07
1,20,1.211905e+07
2,30,4.470917e+07
3,40,1.665006e+07
4,50,8.475490e+05
5,60,2.858194e+07
6,70,8.461224e+06
7,80,3.219519e+07
8,90,4.268191e+06
9,95,1.941690e+05


In [6]:
esa_labels = list(geemap.builtin_legends['ESA_WorldCover'].keys())
esa_labels

['10 Trees',
 '20 Shrubland',
 '30 Grassland',
 '40 Cropland',
 '50 Built-up',
 '60 Barren / sparse vegetation',
 '70 Snow and ice',
 '80 Open water',
 '90 Herbaceous wetland',
 '95 Mangroves',
 '100 Moss and lichen']

In [7]:
df['label'] = esa_labels
df

,key,value,label
0,10,6.139810e+07,10 Trees
1,20,1.211905e+07,20 Shrubland
2,30,4.470917e+07,30 Grassland
3,40,1.665006e+07,40 Cropland
4,50,8.475490e+05,50 Built-up
5,60,2.858194e+07,60 Barren / sparse vegetation
6,70,8.461224e+06,70 Snow and ice
7,80,3.219519e+07,80 Open water
8,90,4.268191e+06,90 Herbaceous wetland
9,95,1.941690e+05,95 Mangroves


In [9]:
round(df['value'].sum() / 1e6, 6)

217.060401

In [11]:
geemap.bar_chart(
    df, x = 'label', y = 'value', x_label = 'Land Cover Type', y_label = 'Area (km2)'
)

In [12]:
geemap.pie_chart(df, names = 'label', values = 'value', height = 500)

## Adding Administrative Boundaries

In [14]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
africa = countries.filter(ee.Filter.eq('CONTINENT', 'Africa'))
style = {'fillColor': '00000000'}
a_style = {'fillColor': 'FFFFFF'}

Map.addLayer(countries.style(**style), {}, 'Countries', False)
Map.addLayer(africa.style(**a_style), {}, 'Africa')
Map.centerObject(africa)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## Extracting Croplands

In [16]:
cropland = esa.eq(40).clipToCollection(africa).selfMask()

Map.addLayer(cropland, {'palette': ['f096ff']}, 'Cropland')
Map.show_layer(name = 'ESA Land Cover', show = False)

## Zonal Statistics

In [17]:
geemap.zonal_stats(
    cropland, africa, 'data/esa_cropland.csv', stat_type = 'SUM', scale = 100
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\esa_cropland.csv


In [18]:
esa_cropland = geemap.csv_to_df('data/esa_cropland.csv')
esa_cropland

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,8.323261e+05,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,4.210671e+06,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,5.785698e+04,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,1.644275e+07,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,8.592826e+05,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia
5,2.085683e+06,152700.0,KE,15,KEN,Africa,47615739,5. Low income,Eastern Africa,0000000000000000000d,Kenya
6,4.687889e+06,28330.0,ZW,14,ZWE,Africa,13805084,5. Low income,Eastern Africa,00000000000000000030,Zimbabwe
7,3.876922e+06,21200.0,MW,14,MWI,Africa,19196246,5. Low income,Eastern Africa,00000000000000000047,Malawi
8,4.725092e+06,35010.0,MZ,15,MOZ,Africa,26573706,5. Low income,Eastern Africa,00000000000000000048,Mozambique
9,8.240984e+05,7892.0,BI,14,BDI,Africa,11466756,5. Low income,Eastern Africa,0000000000000000004b,Burundi


In [21]:
geemap.bar_chart(
    esa_cropland, x = 'NAME', y = 'sum', max_rows = 30, x_label = 'Country', y_label = 'Area (km2)'
)

In [22]:
geemap.pie_chart(
    esa_cropland, names = 'NAME', values = 'sum', max_rows = 20, height = 500
)

# ESRI Global Land Cover

The ESRI Global Land Cover dataset is a global map of LULC derived from ESA Sentinel-2 imagery at 10m resolution. Each year is generated from Impact Observatory's deep learning AI land classificaiton model, which used a massive training dataset made of billions of human-labeled image pixels developed by the National Geographic Society. The global maps were produced by applying this model to the Sentinel-2 scene collection, processing over 400,000 Earth observations per year.
- https://livingatlas.arcgis.com/landcover/
- https://www.arcgis.com/home/item.html?id=d3da5dd386d140cf93fc9ecbf8da5e31
- https://samapriya.github.io/awesome-gee-community-datasets/projects/S2TSLULC/

In [53]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

esri = ee.ImageCollection(
    'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS'
)

esri_2017 = esri.filterDate('2017-01-01', '2017-12-31').mosaic()
esri_2018 = esri.filterDate('2018-01-01', '2018-12-31').mosaic()
esri_2019 = esri.filterDate('2019-01-01', '2019-12-31').mosaic()
esri_2020 = esri.filterDate('2020-01-01', '2020-12-31').mosaic()
esri_2021 = esri.filterDate('2021-01-01', '2021-12-31').mosaic()
esri_2022 = esri.filterDate('2022-01-01', '2022-12-31').mosaic()

esri_vis = {
    'min': 1,
    'max': 11,
    'palette': 'esri_lulc'
}

Map.addLayer(esri_2017, esri_vis, 'ESRI LULC 2017')
Map.addLayer(esri_2018, esri_vis, 'ESRI LULC 2018')
Map.addLayer(esri_2019, esri_vis, 'ESRI LULC 2019')
Map.addLayer(esri_2020, esri_vis, 'ESRI LULC 2020')
Map.addLayer(esri_2021, esri_vis, 'ESRI LULC 2021')
Map.addLayer(esri_2022, esri_vis, 'ESRI LULC 2022')

Map.add_legend(title = 'ESRI Land Cover', builtin_legend = 'ESRI_LandCover_TS')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Time Series Inspector

In [57]:
images = ee.List([esri_2017, esri_2018, esri_2019, esri_2020, esri_2021, esri_2022])
collection = ee.ImageCollection.fromImages(images)
images

In [58]:
years = [str(year) for year in range (2017, 2022 + 1)]
years

['2017', '2018', '2019', '2020', '2021', '2022']

In [59]:
Map = geemap.Map()
Map.ts_inspector(collection, years, esri_vis, width = '80px')
Map.add_legend(title = 'ESRI Land Cover', builtin_legend = 'ESRI_LandCover_TS')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Layo…

## Extracting Croplands

In [60]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
africa = countries.filter(ee.Filter.eq('CONTINENT', 'Africa'))

cropland_col = collection.map(lambda img: img.eq(5).clipToCollection(africa).selfMask())
cropland_ts = cropland_col.toBands().rename(years)
cropland_ts

In [61]:
Map = geemap.Map()

style = {'fillColor': '00000000'}
Map.addLayer(countries.style(**style), {}, 'Countries', False)
Map.addLayer(africa.style(**a_style), {}, 'Africa')
Map.addLayer(cropland_col.first(), {'palette': ['#ab6c28']}, 'first')
Map.centerObject(africa)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [62]:
cropland_ts.bandNames().getInfo()

['2017', '2018', '2019', '2020', '2021', '2022']

## Zonal Statistics

In [63]:
geemap.zonal_stats(
    cropland_ts, africa, 'data/esri_cropland.csv', stat_type = 'SUM', scale = 1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\esri_cropland.csv


In [64]:
esri_cropland = geemap.csv_to_df('data/esri_cropland.csv')
esri_cropland

,2017,2018,2019,2020,2021,2022,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,6900.839216,8345.709804,6293.945098,6026.882353,7079.513725,7741.666667,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,19867.694118,17239.431373,16109.882353,19053.890196,19242.509804,21446.192157,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,367.447059,518.882353,584.901961,717.356863,684.133333,1123.403922,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,46288.988235,45339.407843,50564.792157,55603.462745,58286.588235,65641.952941,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,6913.113725,8625.537255,8530.376471,11638.392157,9955.882353,7949.270588,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia
5,51950.419608,54464.666667,52557.137255,51214.956863,51085.756863,53167.968627,152700.0,KE,15,KEN,Africa,47615739,5. Low income,Eastern Africa,0000000000000000000d,Kenya
6,19770.349020,22166.964706,29371.686275,22390.682353,19896.317647,23081.074510,28330.0,ZW,14,ZWE,Africa,13805084,5. Low income,Eastern Africa,00000000000000000030,Zimbabwe
7,13378.388235,12649.905882,13557.635294,15109.921569,15751.917647,18469.337255,21200.0,MW,14,MWI,Africa,19196246,5. Low income,Eastern Africa,00000000000000000047,Malawi
8,11364.996078,9807.301961,11458.160784,13920.403922,12933.835294,15343.780392,35010.0,MZ,15,MOZ,Africa,26573706,5. Low income,Eastern Africa,00000000000000000048,Mozambique
9,1124.882353,2266.713725,1711.686275,1685.176471,2578.898039,3778.184314,7892.0,BI,14,BDI,Africa,11466756,5. Low income,Eastern Africa,0000000000000000004b,Burundi


In [65]:
geemap.bar_chart(
    esri_cropland, x = 'NAME', y = years, max_rows = 20, legend_title = 'Years'
)

In [66]:
geemap.pie_chart(esri_cropland, names = 'NAME', values = '2020', max_rows = 20, height = 500)

## Analyzing Cropland Gain and Loss

In [67]:
cropland_2017

In [68]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

cropland_2017 = esri_2017.eq(5).selfMask()
cropland_2022 = esri_2022.eq(5).selfMask()

cropland_gain = esri_2017.neq(5).And(esri_2022.eq(5)).selfMask()
cropland_loss = esri_2017.eq(5).And(esri_2022.neq(5)).selfMask()

Map.addLayer(cropland_2017, {'palette': 'brown'}, 'Cropland 2017', False)
Map.addLayer(cropland_2022, {'palette': 'cyan'}, 'Cropland 2022', False)

Map.addLayer(cropland_gain, {'palette': 'yellow'}, 'Cropland gain')
Map.addLayer(cropland_loss, {'palette': 'red'}, 'Cropland loss')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [69]:
geemap.zonal_stats(
    cropland_gain, countries, 'data/esri_cropland_gain.csv', stat_type = 'SUM', scale = 1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\esri_cropland_gain.csv


In [70]:
esri_cropland_gain = geemap.csv_to_df('data/esri_cropland_gain.csv')
esri_cropland_gain

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,1109.745098,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,2711.709804,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,5546.215686,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,8697.317647,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,962.274510,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal
...,...,...,...,...,...,...,...,...,...,...,...
171,4779.968627,182400.0,EC,14,ECU,South America,16290913,3. Upper middle income,South America,0000000000000000002c,Ecuador
172,14011.933333,78350.0,BO,14,BOL,South America,11138234,4. Lower middle income,South America,0000000000000000001e,Bolivia
173,174.619608,6093.0,GY,11,GUY,South America,737718,4. Lower middle income,South America,00000000000000000029,Guyana
174,28908.964706,64670.0,PY,13,PRY,South America,6943739,4. Lower middle income,South America,0000000000000000009b,Paraguay


In [71]:
geemap.bar_chart(
    esri_cropland_gain, x = 'NAME', y = 'sum', max_rows = 30, x_label = 'Country', y_label = 'Area (km2)', title = 'Cropland Gain'
)

In [72]:
geemap.pie_chart(
    esri_cropland_gain, names = 'NAME', values = 'sum', max_rows = 30, height = 500, title = 'Cropland Gain'
)

In [73]:
geemap.zonal_stats(
    cropland_loss, countries, 'data/esri_cropland_loss.csv', stat_type = 'SUM', scale = 1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\gilramolete\OneDrive - UNIONBANK of the Philippines\Documents 1\geemap\data\esri_cropland_loss.csv


In [74]:
esri_cropland_loss = geemap.csv_to_df('data/esri_cropland_loss.csv')
esri_cropland_loss

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,1182.800000,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,1746.988235,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,4194.968627,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,9902.058824,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,1868.317647,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal
...,...,...,...,...,...,...,...,...,...,...,...
171,4072.156863,182400.0,EC,14,ECU,South America,16290913,3. Upper middle income,South America,0000000000000000002c,Ecuador
172,4173.803922,78350.0,BO,14,BOL,South America,11138234,4. Lower middle income,South America,0000000000000000001e,Bolivia
173,360.482353,6093.0,GY,11,GUY,South America,737718,4. Lower middle income,South America,00000000000000000029,Guyana
174,9498.458824,64670.0,PY,13,PRY,South America,6943739,4. Lower middle income,South America,0000000000000000009b,Paraguay


In [75]:
geemap.bar_chart(
    esri_cropland_loss, x = 'NAME', y = 'sum', max_rows = 30, x_label = 'Country', y_label = 'Area (km2)', title = 'Cropland Loss'
)

In [76]:
geemap.pie_chart(
    esri_cropland_loss, names = 'NAME', values = 'sum', max_rows = 30, height = 500, title = 'Cropland Loss'
)

# Dynamic World Land Cover

This is a near realtime 10m resolution global LULC dataset, produced using deep learning, freely available and openly licensed. This is a result of leveraging a novel DL approach, based on Sentinel-2 Top of Atmosphere. This offers global land cover updating every 2-5 days, depending on the location.

## Classification and Probability

In [77]:
Map = geemap.Map()

region = ee.Geometry.BBox(-179, -89, 179, 89)
start_date = '2022-01-01'
end_date = '2023-01-01'

dw_class = geemap.dynamic_world(region, start_date, end_date, return_type = 'class')
dw = geemap.dynamic_world(region, start_date, end_date, return_type = 'hillshade')

dw_vis = {
    'min': 0,
    'max': 8,
    'palette': 'dw'
}

Map.addLayer(dw_class, dw_vis, 'DW Land Cover', False)
Map.addLayer(dw, {}, 'DW Land Cover Hillshade') 

Map.add_legend(title = 'Dynamic World Land Cover', builtin_legend = 'Dynamic_World')
Map.set_center(-88.9088, 43.0006, 12)
Map

Map(center=[43.0006, -88.9088], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## ESA Land Cover vs. Dynamic World

In [78]:
Map = geemap.Map(center = [39.3322, -106.7349], zoom = 10)

left_layer = geemap.ee_tile_layer(esa, esa_vis, "ESA Land Cover")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World Land Cover")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESA Land Cover", builtin_legend='ESA_WorldCover', position='bottomleft'
)
Map.add_legend(
    title = "Dynamic World Land Cover",
    builtin_legend = 'Dynamic_World',
    position = 'bottomright',
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## ESRI Land Cover vs. Dynamic World

In [79]:
Map = geemap.Map(center = [-89.3998, 43.0886], zoom = 10)

left_layer = geemap.ee_tile_layer(esri_2021, esri_vis, "ESRI Land Cover")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World Land Cover")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title = "ESRI Land Cover", builtin_legend = 'ESRI_LandCover', position = 'bottomleft'
)
Map.add_legend(
    title = "Dynamic World Land Cover",
    builtin_legend = 'Dynamic_World',
    position = 'bottomright',
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…